Observations

2. hashtags/punctuations
3. "not worth"/"did not taste bad" —> extracted "worth"/"bad" so meaning changed (negspacy)
4. emojis —> label if it is a good one?
5. quite alot of duplicated rows


TODO : 
1. Extract frequently occuring words with aspects1/2 
2. Send preprocessed data

In [6]:
import pandas as pd
import numpy as np

In [15]:
#path = "/Users/xinminaw/Documents/GitHub/bt4222-burpple-reviews/data/labelled_data/"
path = ""
test = pd.read_csv(path + "test_subset.csv")
train_1 = pd.read_csv(path + "train_first_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]
train_2 = pd.read_csv(path + "train_second_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]
train_3 = pd.read_csv(path + "train_third_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]
train_1["label"] = pd.to_numeric(train_1["label"], errors='coerce')
train = pd.concat([train_1,train_2,train_3],axis=0)
val = pd.read_csv(path + "val_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]

test = test.reset_index().drop("index",axis=1)
train=train.reset_index().drop("index",axis=1)
val = val.reset_index().drop("index",axis=1)

train.head()

,restaurant_code,review_title,review_body,account_name,account_id,aspect,phrase,label,new_aspect_1,new_aspect_2
0,2lhaZp7B,So glad the weekend is here and I can finally ...,Breakfast with the husband this morning at Gen...,Maureen Ow,@72128,food,gen shu,0.0,food,NaN
1,2lhaZp7B,So glad the weekend is here and I can finally ...,Breakfast with the husband this morning at Gen...,Maureen Ow,@72128,food,and glutinous rice,0.0,food,NaN
2,2lhaZp7B,My Saturday morning started with breakfast at ...,It is so inspiring talking to Uncle Kun as he ...,Maureen Ow,@72128,food,it is,0.0,NaN,NaN
3,2lhaZp7B,My Saturday morning started with breakfast at ...,It is so inspiring talking to Uncle Kun as he ...,Maureen Ow,@72128,food,"cantonese style congee ,",0.0,food,NaN
4,2lhaZp7B,Wonderful breakfast to start my day!,Wonderful consistency in texture too！#根叔美食世家 #...,Benjamin Yeo,@4898,food,# 皮蛋粥 # breakfast # porridge,0.0,food,NaN


In [16]:
test.to_csv("test_old.csv")
train.to_csv("train_old.csv")
val.to_csv("val_old.csv")

In [40]:
for data in [train_1,train_2,train_3,train,val,test]:
    print("Total size = {}".format(data.shape[0]))
    print("Noise = {} ".format(data.loc[(data.label == 0.0) & (data.new_aspect_1.isnull()) & (data.new_aspect_2.isnull())].shape[0]))
    print("Aspect with no adjectives = {} ".format(data.loc[(data.new_aspect_1.notnull()) & (data.new_aspect_2.isnull())].shape[0]))
    print("Aspect with adjectives = {} ".format(data.loc[(data.new_aspect_1.notnull()) & (data.new_aspect_2.notnull())].shape[0]))
    print("")

Total size = 1362
Noise = 257 
Aspect with no adjectives = 812 
Aspect with adjectives = 285 

Total size = 1362
Noise = 317 
Aspect with no adjectives = 474 
Aspect with adjectives = 496 

Total size = 1363
Noise = 419 
Aspect with no adjectives = 474 
Aspect with adjectives = 469 

Total size = 4087
Noise = 993 
Aspect with no adjectives = 1760 
Aspect with adjectives = 1250 

Total size = 1497
Noise = 347 
Aspect with no adjectives = 579 
Aspect with adjectives = 569 

Total size = 1472
Noise = 462 
Aspect with no adjectives = 493 
Aspect with adjectives = 509 



##### EDA 

In [94]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
import nltk
from nltk.corpus import stopwords
stopwords1 = stopwords.words('english')
stopwords1.extend(["ambience","price","food","service","portion","time"])
from nltk.stem import WordNetLemmatizer
stemmer = WordNetLemmatizer()
import re
from tqdm import tqdm

In [95]:
# Text cleaning function for gensim fastText word embeddings in python
# remove numbers
def process_text(document):     
        # Remove extra white space from text
        document = re.sub(r'\s+', ' ', document, flags=re.I)     
        # Remove all the special characters from text
        document = re.sub(r'\W', ' ', str(document))
        # Remove all single characters from text
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
        # Converting to Lowercase
        document = document.lower()
        # Word tokenization       
        tokens = document.split()
        # Lemmatization using NLTK
        lemma_txt = [stemmer.lemmatize(word) for word in tokens]
        # Remove stop words
        lemma_no_stop_txt = [word for word in lemma_txt if word not in stopwords1]
        # Drop words 
        tokens = [word for word in tokens if len(word) > 3]            
        clean_txt = ' '.join(lemma_no_stop_txt)
        return clean_txt

In [97]:
from collections import Counter

all_text = pd.concat([train_1,train_2,train_3,val,test],axis=0)
all_text1 = all_text.loc[(all_text.new_aspect_1.notnull()) & (all_text.new_aspect_2.notnull())]
all_text2 = all_text1.groupby(["new_aspect_2"]).agg({"phrase": " ".join})
all_text2 = all_text2.reset_index()
all_text2.phrase = all_text2.phrase.map(process_text)
count = 0 

all_text2.new_aspect_2.values


data_set = all_text2.loc[all_text2.new_aspect_2 == "ambience" , "phrase"][0]
# split() returns list of all the words in the string
split_it = data_set.split()

# Pass the split_it list to instance of Counter class.
Counter0 = Counter(split_it)
#print(Counter)

# most_common() produces k frequently encountered
# input values and their respective counts.
most_occur = Counter0.most_common(8)
print(most_occur)

data_set = all_text2.loc[all_text2.new_aspect_2 == "food" , "phrase"][1]
# split() returns list of all the words in the string
split_it = data_set.split()

# Pass the split_it list to instance of Counter class.
Counter1 = Counter(split_it)
#print(Counter)

# most_common() produces k frequently encountered
# input values and their respective counts.
most_occur = Counter1.most_common(8)
print(most_occur)



data_set = all_text2.loc[all_text2.new_aspect_2 == "portion" , "phrase"][2]
# split() returns list of all the words in the string
split_it = data_set.split()

# Pass the split_it list to instance of Counter class.
Counter2 = Counter(split_it)
#print(Counter)

# most_common() produces k frequently encountered
# input values and their respective counts.
most_occur = Counter2.most_common(8)
print(most_occur)

data_set = all_text2.loc[all_text2.new_aspect_2 == "price" , "phrase"][3]
# split() returns list of all the words in the string
split_it = data_set.split()

# Pass the split_it list to instance of Counter class.
Counter3 = Counter(split_it)
#print(Counter)

# most_common() produces k frequently encountered
# input values and their respective counts.
most_occur = Counter3.most_common(8)
print(most_occur)

data_set = all_text2.loc[all_text2.new_aspect_2 == "service" , "phrase"][4]
# split() returns list of all the words in the string
split_it = data_set.split()

# Pass the split_it list to instance of Counter class.
Counter4 = Counter(split_it)
#print(Counter)

# most_common() produces k frequently encountered
# input values and their respective counts.
most_occur = Counter4.most_common(8)
print(most_occur)

data_set = all_text2.loc[all_text2.new_aspect_2 == "time" , "phrase"][5]
# split() returns list of all the words in the string
split_it = data_set.split()

# Pass the split_it list to instance of Counter class.
Counter5 = Counter(split_it)
#print(Counter)

# most_common() produces k frequently encountered
# input values and their respective counts.
most_occur = Counter5.most_common(8)
print(most_occur)

[('wa', 10), ('great', 8), ('place', 8), ('nice', 6), ('atmosphere', 6), ('good', 4), ('sea', 3), ('view', 3)]
[('wa', 205), ('good', 118), ('sauce', 77), ('sweet', 73), ('taste', 65), ('noodle', 54), ('fresh', 52), ('tender', 51)]
[('generous', 57), ('small', 28), ('serving', 24), ('wa', 18), ('big', 17), ('size', 16), ('amount', 16), ('good', 13)]
[('affordable', 19), ('worth', 16), ('cheaper', 11), ('charge', 8), ('reasonable', 8), ('tag', 7), ('value', 7), ('money', 6)]
[('wa', 14), ('friendly', 13), ('good', 8), ('great', 6), ('staff', 5), ('excellent', 3), ('attentive', 3), ('bad', 3)]
[('long', 18), ('queue', 17), ('wait', 8), ('lunch', 6), ('even', 6), ('waiting', 5), ('snaking', 4), ('dinner', 4)]


##### Preprocessing

2 data sets for modelling

1. Use this original data without noise
2. Create new dataset --> Filter the phrase column (punctuations/hashtags) then combine the aspects into 1 & aggregate the sentiment scores (label)

Question : when aggregating use majority? but what if i had alot of noise eg. 0 0 0 0 0 and only 1 -1 then using majority voting will only get 0 not -1? And also mean doesnt work either

In [41]:
# train
noise = train.loc[(train.label == 0.0) & (train.new_aspect_1.isnull()) & (train.new_aspect_2.isnull())].index
train1 = train.loc[~train.index.isin(noise)]
print(train1.shape)

# test
noise = val.loc[(val.label == 0.0) & (val.new_aspect_1.isnull()) & (val.new_aspect_2.isnull())].index
val1 = val.loc[~val.index.isin(noise)]
print(val1.shape)

#val 
noise = test.loc[(test.label == 0.0) & (test.new_aspect_1.isnull()) & (test.new_aspect_2.isnull())].index
test1 = test.loc[~test.index.isin(noise)]
print(test1.shape)

(3094, 11)
(1150, 11)
(1010, 11)


In [42]:
print(train1.label.value_counts())
print(test1.label.value_counts())
print(val1.label.value_counts())

 0.0    2025
 1.0     897
-1.0     171
Name: label, dtype: int64
 0.0    516
 1.0    430
-1.0     63
Name: label, dtype: int64
 0    762
 1    335
-1     53
Name: label, dtype: int64


#### aggregate sentiment again!

- average then sign(>0 = 1,=0 =0 <0 = -1)

In [43]:
# filter out phrases with hashtags & those with character length <=5 then get the majority (might not make sense)
# train
train2_filtered = train.loc[~train.index.isin(train.loc[(train.phrase.str.contains("#")) | (train.phrase.str.len() <= 5)].index)]
print("Number of rows after filtering (Train) = {}".format(train2_filtered.shape[0]))
train2 = train2_filtered.groupby(["review_title","account_name","aspect"]).agg({"phrase": " ".join,
                                                                                "label": lambda x:x.value_counts().index[0]})
train2 = train2.reset_index()


# test
test2_filtered = test.loc[~test.index.isin(test.loc[(test.phrase.str.contains("#")) | (test.phrase.str.len() <= 5)].index)]
print("Number of rows after filtering (Test) = {}".format(test2_filtered.shape[0]))
test2 = test2_filtered.groupby(["review_title","account_name","aspect"]).agg({"phrase": " ".join,
                                                                              "label": lambda x:x.value_counts().index[0]})
test2 = test2.reset_index()

#val 
val2_filtered = val.loc[~val.index.isin(val.loc[(val.phrase.str.contains("#")) | (val.phrase.str.len() <= 5)].index)]
print("Number of rows after filtering (Val) = {}".format(val2_filtered.shape[0]))
val2 = val2_filtered.groupby(["review_title","account_name","aspect"]).agg({"phrase": " ".join,
                                                                              "label": lambda x:x.value_counts().index[0]})
val2 = val2.reset_index()

Number of rows after filtering (Train) = 3813
Number of rows after filtering (Test) = 1352
Number of rows after filtering (Val) = 1388


In [44]:
# use train & val as training
train_val2 = pd.concat([train2,val2],axis=0)
train_val2.shape

(1639, 5)

In [45]:
print(train_val2.label.value_counts())
print(test2.label.value_counts())

 0.0    1120
 1.0     434
-1.0      85
Name: label, dtype: int64
 0.0    263
 1.0    133
-1.0     21
Name: label, dtype: int64


###### Text Vectorisation --> used on data2
1. BoW
2. Tf-idf
3. Fasttext - can handle oov words https://www.quora.com/How-does-fastText-output-a-vector-for-a-word-that-is-not-in-the-pre-trained-model
4. word2vec


FastText vs word2vec
Word2vec treats each word like an atomic entity and generates a vector for each word. Word2vec cannot provide good results for rare and out of vocabulary words.

FastText (an extension of word2vec model), treats each word as composed of character n-grams. FastText word embeddings generate better word embeddings for rare and out of vocabulary words because even if words are rare their character n-grams are still shared with other words.

In [55]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
import nltk
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
from nltk.stem import WordNetLemmatizer
stemmer = WordNetLemmatizer()
import re
from tqdm import tqdm

In [56]:
# Text cleaning function for gensim fastText word embeddings in python
# remove numbers
def process_text(document):     
        # Remove extra white space from text
        document = re.sub(r'\s+', ' ', document, flags=re.I)     
        # Remove all the special characters from text
        document = re.sub(r'\W', ' ', str(document))
        # Remove all single characters from text
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
        # Converting to Lowercase
        document = document.lower()
        # Word tokenization       
        tokens = document.split()
        # Lemmatization using NLTK
        lemma_txt = [stemmer.lemmatize(word) for word in tokens]
        # Remove stop words
        lemma_no_stop_txt = [word for word in lemma_txt if word not in stopwords]
        # Drop words 
        tokens = [word for word in tokens if len(word) > 3]            
        clean_txt = ' '.join(lemma_no_stop_txt)
        return clean_txt

In [57]:
bow = CountVectorizer(analyzer='word',lowercase=True,ngram_range=(1, 3),preprocessor=process_text)
bow_train = bow.fit_transform(train_val2.phrase)
bow_test = bow.transform(test2.phrase)

In [30]:
tfidf = TfidfVectorizer(analyzer='word',lowercase=True,ngram_range=(1, 3),preprocessor=process_text)
tfidf_train = tfidf.fit_transform(train_val2.phrase)
tfidf_test = tfidf.transform(test2.phrase)

In [31]:
from gensim.models.fasttext import FastText
from gensim.models import Word2Vec

# Preprocess data for fasttext word embeddings training
train_clean = [process_text(sentence) for sentence in tqdm(train_val2.phrase) if sentence.strip() !='']
word_tokenizer = nltk.WordPunctTokenizer()
train_word_tokens = [word_tokenizer.tokenize(sent) for sent in tqdm(train_clean)]

# Preprocess data for fasttext word embeddings testing data
test_clean = [process_text(sentence) for sentence in tqdm(test2.phrase) if sentence.strip() !='']
word_tokenizer = nltk.WordPunctTokenizer()
test_word_tokens = [word_tokenizer.tokenize(sent) for sent in tqdm(test_clean)]

# Train Fasttext word embeddings on training data
# Defining values for parameters
embedding_size = 300
window_size = 5
min_word = 5
down_sampling = 1e-2
 

fast_Text_model = FastText(train_word_tokens,
                      size=embedding_size,
                      window=window_size,
                      min_count=min_word,
                      sample=down_sampling,
                      workers = 4,
                      sg=1,
                      iter=100)

100%|██████████| 417/417 [00:00<00:00, 95247.22it/s]


In [32]:
# Save fastText gensim model
fast_Text_model.save("ftmodel/ft_model_train")
# Load saved gensim fastText model
fast_Text_model = Word2Vec.load("ftmodel/ft_model_train")

In [33]:
# convert sentence --> average(each word embedding)
fasttext_train = [np.array([fast_Text_model[word] for word in sentence]).mean(axis=0) for sentence in train_word_tokens]
# convert sentence --> average(each word embedding)
fasttext_test = [np.array([fast_Text_model[word] for word in sentence]).mean(axis=0) for sentence in test_word_tokens]


<ipython-input-33-0e7f7a7a4025>:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fasttext_train = [np.array([fast_Text_model[word] for word in sentence]).mean(axis=0) for sentence in train_word_tokens]
<ipython-input-33-0e7f7a7a4025>:2: RuntimeWarning: Mean of empty slice.
  fasttext_train = [np.array([fast_Text_model[word] for word in sentence]).mean(axis=0) for sentence in train_word_tokens]
/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-33-0e7f7a7a4025>:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  fasttext_test = [np.array([fast_Text_model[word] for word in sentence]).mean(axis=0) for sentence in test_word_tokens]
<ipython-input-33-0e7f7a7a4025>:4: RuntimeWarning: Mean of emp

In [34]:
fasttext_train_processed = []
for row in fasttext_train:
    try:
        fasttext_train_processed.append(list(row))
    except:
        fasttext_train_processed.append([0]*300)

fasttext_test_processed = []
for row in fasttext_test:
    try:
        fasttext_test_processed.append(list(row))
    except:
        fasttext_test_processed.append([0]*300)



In [35]:
fasttext_train_processed = np.array(fasttext_train_processed).reshape(-1,300)
fasttext_test_processed = np.array(fasttext_test_processed).reshape(-1,300)

In [17]:
from gensim.models import Word2Vec

##### Modelling

1. Naive Bayes/Support Vector Machines (NBSVM)/Logistic Regression
2. Fasttext
3. BERT 


(Baseline) 1. Naive Bayes -Support Vector Machines (NBSVM)- Logistic Regression
- works very well when the dataset is very small, at times it worked better than the neural networks based models. Sida Wang et al. 
- showed that the simple NB and SVM variants outperformed most published results on several sentiment analysis datasets (snippets and longer documents) sometimes providing a new state-of-the-art performance level.
- A simple and efficient baseline for sentence classification is to represent sentences as bag of words (BoW) and train a linear classifier, e.g., a logistic regression or an SVM


In [36]:
# import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.svm import SVC

logreg = LogisticRegression()
nb = MultinomialNB()
svm = SVC()

all_train = [bow_train,tfidf_train,fasttext_train_processed]
all_test = [bow_test,tfidf_test,fasttext_test_processed]
all_vectorisers = ["BoW","TfIdf","FastText"]

for train,test,vectoriser in zip(all_train,all_test,all_vectorisers):
    logreg.fit(train,train_val2.label)
    
    print("Logistic Regression + " + vectoriser)
    print("Training Data")
    print(classification_report(train_val2.label, logreg.predict(train)))
    print("Test Data")
    print(classification_report(test2.label, logreg.predict(test))) 
    
    svm.fit(train,train_val2.label)
    
    print("SVM + " + vectoriser)
    print("Training Data")
    print(classification_report(train_val2.label, svm.predict(train)))
    print("Test Data")
    print(classification_report(test2.label, svm.predict(test))) 

    
    nb.fit(train,train_val2.label)
    print("Naive Bayes + "+ vectoriser)
    print("Training Data")
    print(classification_report(train_val2.label, nb.predict(train)))
    print("Test Data")
    print(classification_report(test2.label, nb.predict(test)))    

Logistic Regression + BoW
Training Data
              precision    recall  f1-score   support

        -1.0       1.00      0.92      0.96        85
         0.0       0.99      1.00      1.00      1120
         1.0       1.00      0.99      0.99       434

    accuracy                           0.99      1639
   macro avg       1.00      0.97      0.98      1639
weighted avg       0.99      0.99      0.99      1639

Test Data
              precision    recall  f1-score   support

        -1.0       0.67      0.10      0.17        21
         0.0       0.68      0.89      0.77       263
         1.0       0.53      0.29      0.37       133

    accuracy                           0.66       417
   macro avg       0.63      0.42      0.44       417
weighted avg       0.63      0.66      0.61       417

SVM + BoW
Training Data
              precision    recall  f1-score   support

        -1.0       1.00      0.13      0.23        85
         0.0       0.84      1.00      0.91      1120
 

/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Logistic Regression + TfIdf
Training Data
              precision    recall  f1-score   support

        -1.0       1.00      0.01      0.02        85
         0.0       0.82      1.00      0.90      1120
         1.0       0.99      0.61      0.75       434

    accuracy                           0.84      1639
   macro avg       0.93      0.54      0.56      1639
weighted avg       0.87      0.84      0.81      1639

Test Data
              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00        21
         0.0       0.68      0.97      0.80       263
         1.0       0.71      0.22      0.33       133

    accuracy                           0.68       417
   macro avg       0.46      0.40      0.38       417
weighted avg       0.66      0.68      0.61       417



/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


SVM + TfIdf
Training Data
              precision    recall  f1-score   support

        -1.0       1.00      0.47      0.64        85
         0.0       0.96      1.00      0.98      1120
         1.0       1.00      0.99      1.00       434

    accuracy                           0.97      1639
   macro avg       0.99      0.82      0.87      1639
weighted avg       0.97      0.97      0.97      1639

Test Data
              precision    recall  f1-score   support

        -1.0       1.00      0.05      0.09        21
         0.0       0.67      0.99      0.80       263
         1.0       0.73      0.14      0.24       133

    accuracy                           0.67       417
   macro avg       0.80      0.39      0.38       417
weighted avg       0.70      0.67      0.58       417

Naive Bayes + TfIdf
Training Data
              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00        85
         0.0       0.78      1.00      0.87      1120
     

/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


SVM + FastText
Training Data
              precision    recall  f1-score   support

        -1.0       1.00      0.13      0.23        85
         0.0       0.78      0.99      0.87      1120
         1.0       0.91      0.46      0.61       434

    accuracy                           0.80      1639
   macro avg       0.90      0.52      0.57      1639
weighted avg       0.83      0.80      0.77      1639

Test Data
              precision    recall  f1-score   support

        -1.0       1.00      0.10      0.17        21
         0.0       0.66      0.98      0.79       263
         1.0       0.75      0.16      0.26       133

    accuracy                           0.67       417
   macro avg       0.80      0.41      0.41       417
weighted avg       0.71      0.67      0.59       417



ValueError: Negative values in data passed to MultinomialNB (input X)

2. BERT

- Transfer Learning in NLP

Transfer learning is a technique where a deep learning model trained on a large dataset is used to perform similar tasks on another dataset. We call such a deep learning model a pre-trained model. The most renowned examples of pre-trained models are the computer vision deep learning models trained on the ImageNet dataset. So, it is better to use a pre-trained model as a starting point to solve a problem rather than building a model from scratch.

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

2. Fasttext - FastText uses a simple and efficient baseline for sentence classification( represent sentences as bag of words (BoW) and train a linear classifier). It uses negative sampling, hierarchical softmax and N-gram features to reduce computational cost and improve efficiency.

In [ ]:
import fasttext

In [ ]:
#data preparation for fasttext
with open('fasttext_input_train2.txt', 'w') as f:
    for each_text, each_label in zip(train2['phrase'], train2['label']):
        f.writelines(f'__label__{each_label} {each_text}\n')

In [ ]:
#data preparation for fasttext
with open('fasttext_input_val2.txt', 'w') as f:
    for each_text, each_label in zip(val2['phrase'], val2['label']):
        f.writelines(f'__label__{each_label} {each_text}\n')

In [ ]:
#data preparation for fasttext
with open('fasttext_input_test2.txt', 'w') as f:
    for each_text, each_label in zip(test2['phrase'], test2['label']):
        f.writelines(f'__label__{each_label} {each_text}\n')

In [ ]:
model = fasttext.train_supervised('fasttext_input_train2.txt', wordNgrams=2)

In [ ]:
#measuring performance on test data
def print_results(sample_size, precision, recall):
    precision   = round(precision, 2)
    recall      = round(recall, 2)
    print(f'{sample_size=}')
    print(f'{precision=}')
    print(f'{recall=}')

print_results(*model.test('fasttext_input_test2.txt'))

In [ ]:
model.predict("great service and delectable modern chinese")

Problems
1. Deal with emojis/punctuations